<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Mortage-Default-Prediction" data-toc-modified-id="Mortage-Default-Prediction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Mortage Default Prediction</a></span></li></ul></div>

## Mortage Default Prediction

In [1]:
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-ddd47da4-acfa-4590-babf-ab051b7f77b6',
    'iam_service_endpoint': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'api_key': 'kCxdy2mVdTMcTotMupUlOh_cc6Wrynmh4b7kIAIZ98AU'
}

configuration_name = 'os_f99601c62bd54655b8bdd838cc2ad12e_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(cos.url('Mortgage_Training Data.csv', 'pyrtraining-donotdelete-pr-8zy0j2en6j4jea'))
data.take(5)


ModuleNotFoundError: No module named 'ibmos2spark'

In [3]:
import pandas as pd

In [2]:
pandasDF=data.toPandas()
pandasDF.head()

NameError: name 'data' is not defined

In [13]:
data.dtypes

[('ID', 'double'),
 ('Income', 'double'),
 ('AppliedOnline', 'string'),
 ('Residence', 'string'),
 ('YearCurrentAddress', 'double'),
 ('YearsCurrentEmployer', 'double'),
 ('NumberOfCards', 'double'),
 ('CCDebt', 'double'),
 ('Loans', 'double'),
 ('LoanAmount', 'double'),
 ('SalePrice', 'int'),
 ('Location', 'int'),
 ('MortgageDefault', 'string')]

In [18]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorIndexer, IndexToString
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier


SI1 = StringIndexer(inputCol='AppliedOnline', outputCol='AppliedOnlineEncoded')
SI2 = StringIndexer(inputCol='Residence',outputCol='ResidenceEncoded')
SI3 = StringIndexer(inputCol='Location',outputCol='LocationEncoded')
labelIndexer = StringIndexer(inputCol='MortgageDefault', outputCol='label').fit(data)

OH1 = OneHotEncoder(inputCol="AppliedOnlineEncoded", outputCol="AppliedOnlineEncoded"+"classVec")
OH2 = OneHotEncoder(inputCol="ResidenceEncoded", outputCol="ResidenceEncoded"+"classVec")
OH3 = OneHotEncoder(inputCol="LocationEncoded", outputCol="LocationEncoded"+"classVec")

assembler = VectorAssembler(inputCols=["Income", "AppliedOnlineEncoded"+"classVec",\
                                       "ResidenceEncoded"+"classVec", "YearCurrentAddress",\
                                       "YearsCurrentEmployer", "NumberOfCards", "CCDebt", \
                                       "Loans", "LoanAmount", "SalePrice", "LocationEncoded"+"classVec"],\
                            outputCol="features")

In [19]:
rf=RandomForestClassifier(labelCol="label", featuresCol="features")

labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labelIndexer.labels)

pipeline = Pipeline(stages=[SI1,SI2,SI3,labelIndexer,OH1,OH2,OH3,assembler,rf,labelConverter])

In [20]:
train, test = data.randomSplit([0.8,0.2], seed=6)
train.cache()
test.cache()

DataFrame[ID: double, Income: double, AppliedOnline: string, Residence: string, YearCurrentAddress: double, YearsCurrentEmployer: double, NumberOfCards: double, CCDebt: double, Loans: double, LoanAmount: double, SalePrice: int, Location: int, MortgageDefault: string]

In [21]:
model = pipeline.fit(train)

In [27]:
results=model.transform(test)
results.printSchema()

root
 |-- ID: double (nullable = true)
 |-- Income: double (nullable = true)
 |-- AppliedOnline: string (nullable = true)
 |-- Residence: string (nullable = true)
 |-- YearCurrentAddress: double (nullable = true)
 |-- YearsCurrentEmployer: double (nullable = true)
 |-- NumberOfCards: double (nullable = true)
 |-- CCDebt: double (nullable = true)
 |-- Loans: double (nullable = true)
 |-- LoanAmount: double (nullable = true)
 |-- SalePrice: integer (nullable = true)
 |-- Location: integer (nullable = true)
 |-- MortgageDefault: string (nullable = true)
 |-- AppliedOnlineEncoded: double (nullable = false)
 |-- ResidenceEncoded: double (nullable = false)
 |-- LocationEncoded: double (nullable = false)
 |-- label: double (nullable = false)
 |-- AppliedOnlineEncodedclassVec: vector (nullable = true)
 |-- ResidenceEncodedclassVec: vector (nullable = true)
 |-- LocationEncodedclassVec: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)

In [9]:
level = [1,0,0,0,2]
for i in level:
    print(level.index(i))

0
1
1
1
4
